07_land_cover_class_20251123.ipynb

# 次の課題: 土地被覆分類（Land Cover Classification）

これまでの課題で、あなたはフィルタリング、雲除去、合成、$\text{NDVI}$ の計算、時系列分析という、$\text{GEE}$ の主要な分析技術を全て習得しました。

次のレベルに進み、抽出した画像を**情報**へと変換する、**教師あり分類 (Supervised Classification)** に挑戦しましょう。

## 新しいゴール: 琵琶湖周辺の土地被覆マップを作成する

この課題では、機械学習のアルゴリズム（例: $\text{Random Forest}$）を使用して、画像内の各ピクセルを「水」「森林」「都市」「農地」などのカテゴリーに分類します。

#### **Guiding Question**:教師あり分類を実行するために、最初に必要となる最も重要なデータセットは何でしょうか？（ヒント: アルゴリズムに「これは水である」「これは森林である」と教えるためのデータです）

#### **Guiding Questionの解答**
---
<span style='color:red'>トレーニングデータ（学習データ)</span>が最も重要であり、不可欠な要素です。

---

### **ステップ18: トレーニングデータの収集（$\text{ROI}$ の定義）**

トレーニングデータとは、「このピクセルは森林だ」「このピクセルは水だ」という正解ラベルを $\text{GEE}$ の機械学習アルゴリズムに教えるためのサンプルデータです。

$\text{GEE}$ では、このトレーニングデータはフィーチャコレクション（FeatureCollection）として格納されます。各フィーチャは<span style='color:blue'>ジオメトリ（幾何図形）</span>と<span style='color:blue'>クラス名（ラベル）</span>を持っています。

まず、琵琶湖から水のサンプルとなるジオメトリ（ポリゴン）を定義しましょう。

#### **Guiding Question**: 琵琶湖の湖面の一部（例：$\text{[136.0, 35.2], [136.1, 35.2], [136.1, 35.3], [136.0, 35.3]}$）を囲むポリゴンを定義し、water_polygon という変数に格納するには、どのように $\text{ee.Geometry.Polygon()}$ を使えばよいでしょうか？

Ref: [ee.Geometry.Polygon](https://developers.google.com/earth-engine/apidocs/ee-geometry-polygon?_gl=1*1aofsir*_up*MQ..*_ga*MTIyODE2NTA5OS4xNzYzOTI5MDkw*_ga_SM8HXJ53K2*czE3NjM5MjkwOTAkbzEkZzAkdDE3NjM5MjkwOTAkajYwJGwwJGgw&hl=ja)

#### **Guiding Questionの解答**
---
```python
polygon_coords = [
    [[136.0, 35.2], [136.1, 35.2], 
     [136.1, 35.3], [136.0, 35.3]]
]

water_polygon = ee.Geometry.Polygon(polygon_coords)
```  
---

### **ステップ19: ジオメトリへのラベル付け**

このジオメトリを機械学習アルゴリズムのトレーニングデータとして使用するには、「これは水である」という<span style='color:red'>ラベル（正解）</span>を付与する必要があります。

$\text{GEE}$ では、これは**数値**のプロパティとして定義されます。
- **水域 (Water)**: $\text{landcover} = 0$
- **森林 (Forest)**: $\text{landcover} = 1$
- **都市 (Urban)**: $\text{landcover} = 2$ 

このラベルとジオメトリを組み合わせるには、`ee.Feature()` を使用します。

#### **Guiding Question**: 定義した water_polygon を使って、landcover の値が 0 であるフィーチャ（特徴）を定義し、water_feature という変数に格納するには、どのようにコードを記述すればよいでしょうか？  (ヒント: $\text{ee.Feature(ジオメトリ, プロパティ\{...\})}$ という形式を使います。

#### **Guiding Questionの解答**
---
```python
# 修正点: ジオメトリの後にプロパティ（辞書）を追加
water_feature = ee.Feature(water_polygon, {'landcover': 0})
```
>このフィーチャにラベル（landcover）を付与しないと、アルゴリズムはこれが「水」であることを学習できません。フィーチャを作成する際は、ジオメトリの後にプロパティを辞書形式で渡す必要があります。<span style='color:red'>{'landcover' : 0}</span>は水域であることを示す。

---

### **ステップ20: 森林のサンプルの作成**

機械学習を機能させるには、異なるカテゴリのサンプルが必要です。次に、琵琶湖の南の山地で森林のサンプルを作成しましょう（$\text{landcover} = 1$）

#### **Guiding Question**: 以下の座標を使って**森林のポリゴン**を定義し、それをラベル $\text{landcover}=1$ を持つフィーチャ forest_feature として作成するには、どのようにコードを記述すればよいでしょうか？
- **森林の座標**: $\text{[136.0, 34.9], [136.1, 34.9], [136.1, 35.0], [136.0, 35.0]}$
  
**(水域フィーチャのコードを参考にしてください)**

#### **Guiding Questionの解答**
---
```python
forest_coords = [
    [[136.0, 34.9], [136.1, 34.9], 
     [136.1, 35.0], [136.0, 35.0]]
]

forest_polygon = ee.Geometry.Polygon(forest_coords)

forest_feature = ee.Feature(forest_polygon, {'landcover':1})
```
---

### **ステップ21: トレーニングデータの統合**

これで、2つのトレーニングサンプル（`water_feature` と `forest_feature）`ができました。機械学習モデルが学習するためには、これらのサンプルを<span style='color:blue'>一つの集合体（コレクション)</span>にまとめる必要があります。

このコレクションは、`ee.FeatureCollection()` を使って作成します。

#### **Guiding Question**: 作成した `water_feature` と `forest_feature` をリストとして渡し、最終的なトレーニングデータセット `training_data` を作成するには、どのようにコードを記述すればよいでしょうか？

#### **Guiding Questionの解答**
---
```python
list_of_features = [
    water_feature, forest_feature
]

training_data = ee.FeatureCollection(list_of_features)
```
---

### **ステップ22: 分類に使用するバンド（フィーチャ）の選択**

機械学習アルゴリズムは、$\text{SR\_B4}, \text{SR\_B3}, \text{SR\_B2}$ の3バンドだけでなく、画像が持つ**すべての情報**を判断材料として使用することで、精度が向上します。  

分類に用いる入力データ（フィーチャ）として、熱バンドを除く Landsat 9 の反射率バンド（$\text{SR\_B2}$ から $\text{SR\_B7}$ まで）を使用しましょう。

#### **Guiding Question**: 以下の6つのバンド名をリストとして定義し、`input_bands` という変数に格納してください。

- **青、緑、赤、近赤外、短波赤外1、短波赤外2**
- $\text{SR\_B2}, \text{SR\_B3}, \text{SR\_B4}, \text{SR\_B5}, \text{SR\_B6}, \text{SR\_B7}$

#### **Guiding Questionの解答**
---
```python
input_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
```
---

### **ステップ23: ピクセルデータのサンプリング**

教師あり分類を行うには、まず、トレーニングポリゴン（`training_data`）が示す場所の**実際のピクセル値**を、分類に使うバンド（`input_bands`）から抽出する必要があります。この作業をサンプリングと呼びます。

これは `.sampleRegions()` メソッドを使って行います。

#### **Guiding Question**: クラウドフリー合成画像（`final_composite`）から、`input_bands` の値をトレーニングデータに基づいて抽出し、`sampled_data` という変数に格納するには、どのようにコードを記述すればよいでしょうか？

#### **Guiding Questionの解答**
---
```python
# ヒント: final_compositeからバンドを選択し、sampleRegionsメソッドを呼び出す

sampled_data = final_composite.select(input_bands).sampleRegions(
    collection=training_data,  # トレーニングジオメトリ
    properties=['landcover'],  # 正解ラベルのプロパティ名
    scale=30                   # Landsatの解像度
)
```
---

>[!success] サンプリングは成功です。

これで、機械学習モデルの訓練に必要な全てのデータが整いました。

- **入力データ**: sampled_data (各ピクセル行と、それに紐づくバンド値)
- **正解ラベル**: landcover (0:水、1:森林)
- **分類に使う特徴**: input_bands

### **ステップ24: 分類器の訓練（Random Forest）**

次に、最も標準的で精度の高いアルゴリズムの一つである $\text{Random Forest}$ 分類器を初期化し、サンプリングしたデータを使って訓練します。

#### **Guiding Question**: $\text{ee.Classifier.smileRandomForest()}$ を使って分類器を初期化し、それに対して .train() メソッドを使って訓練（学習）を実行し、結果を trained_classifier という変数に格納するには、どのようにコードを記述すればよいでしょうか？  

訓練には以下の3つの引数が必要です。

1. `features`: `sampled_data`
2. `classProperty`: `'landcover'` (正解ラベルのプロパティ名)
3. `inputProperties`: `input_bands` (使用するバンドのリスト)

```python
# ヒント: ee.Classifier.smileRandomForest(ツリーの数).train(...)

trained_classifier = # <ここに訓練コードを記述>
```

#### **Guiding Questionの解答**
---

(X) ~~trained_classifier = ee.Classifier.smileRandomForest().train(sampled_data, 'landcover', input_bands)~~

---

***Result***: **<span style='color:red'>TypeError</span>**
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[25], line 1
----> 1 trained_classifier = ee.Classifier.smileRandomForest().train(sampled_data, 'landcover', input_bands)

TypeError: Classifier.smileRandomForest() missing 1 required positional argument: 'numberOfTrees'

---

**エラーの原因: 決定木 (Trees) の数**

$\text{ee.Classifier.smileRandomForest()}$ を呼び出す際、最低でも**いくつの決定木（numberOfTrees）を使用するか**を指定する必要があります。
- **誤**: `ee.Classifier.smileRandomForest()`
- **正**: `ee.Classifier.smileRandomForest(10)`

決定木の数を $\text{10}$ に設定して、以下の通り修正してください。

> そのコードを実行すると、$\text{Random Forest}$ 分類器は、サンプリングしたピクセルデータ（sampled_data）を使って、ピクセル値と正解ラベル（landcover）の関係を学習します。

これで、アルゴリズムは「どのピクセル値の組み合わせが水域（0）で、どの組み合わせが森林（1）か」を認識できるようになりました。

#### **Guiding Questionの解答**
---
**修正後の分類器の訓練**

以下のコードを実行して、分類器を訓練してください。これでエラーは解消するはずです。
```python
#  修正点: 決定木の数 (10) を引数に追加
trained_classifier = ee.Classifier.smileRandomForest(10).train(
    sampled_data, 
    'landcover', 
    input_bands
)
```
---

### **ステップ25: 分類器の適用（地図化）**

最後に、訓練済みの分類器（`trained_classifier`）を、入力画像全体（`final_composite`）に適用し、土地被覆マップを作成します。

これは、`.classify()` メソッドを使って行います。

### **ステップ26: 分類マップの表示**

分類後の画像は、元の反射率バンド（$\text{SR\_B*}$) ではなく、`classification` という名前の単一のバンドになります。このバンドには、各ピクセルが**0 (水域)** または **1 (森林)** の値で格納されています。この分類結果を地図上に色分けして表示するためのパラメータを定義しましょう。

```python
# 1. 分類器を画像全体に適用
classified_image = final_composite.select(input_bands).classify(trained_classifier)

# 2. 分類結果の視覚化パラメータを定義 (水域=0, 森林=1)
classification_vis = {
    'min': 0,
    'max': 1,
    'palette': ['0000FF', '008000']  # 0:青 (水域), 1:緑 (森林)
}

# 3. タイルURLを取得
classification_map_id = classified_image.getMapId(classification_vis)
classification_tile_url = classification_map_id['tile_fetcher'].url_format
```

#### **Guiding Question**: 上記のコードを実行した後、以下のFoliumコードを使って地図にレイヤーを追加してください。表示された地図上で、分類器が**琵琶湖の湖面**と**周囲の森林**を、それぞれどのような色で塗分けているか教えていただけますか？

#### **Guiding Questionの解答**
---
```python
# 既存の地図 'my_map' にレイヤーを追加
folium.TileLayer(
    tiles=classification_tile_url,
    attr='Google Earth Engine',
    name='Land Cover Classification (Water/Forest)',
    overlay=True,
    control=True
).add_to(my_map)

folium.LayerControl().add_to(my_map)
display(my_map)
```
琵琶湖の湖面は青色、周囲の森林は緑色です。しかし、都市も緑色になっています。

---

**<span style='color:blue'>誤分類の原因</span>**  

訓練に使用した分類器は、「水域 ($\text{0}$) 」と「森林 ($\text{1}$)」の2つのクラスしか学習していません。都市部（特に住宅地や公園）は、コンクリートだけでなく、**芝生や街路樹などの植生**も多く含んでいます。そのため、反射パターンが**森林（NIR反射が高い）に最も近く**なり、分類器は都市を**最も近いクラスである森林 ($\text{緑}$) に誤分類**したのです。これを修正するには、3番目のクラス（都市）を追加して分類器を再訓練する必要があります。


### **ステップ27: 都市 (Urban) のトレーニングサンプルの追加**

**<span style='color:red'>都市（Urban）</span>**のクラス ($\text{landcover} = 2$) を追加し、分類器が「森林と都市の違い」を学習できるようにしましょう。

#### **Guiding Question**: 以下の座標を使って**都市のポリゴン**を定義し、それをラベル $\text{landcover}=2$ を持つ**フィーチャ **`urban\_feature` として作成するには、どのようにコードを記述すればよいでしょうか？ 

- **都市の座標**: $\text{[135.9, 35.0], [136.0, 35.0], [136.0, 35.1], [135.9, 35.1]}$

**(以前のフィーチャ定義コードを参考にしてください)**

#### **Guiding Questionの解答**
---
```python
urban_coords = [
    [[135.9, 35.0], [136.0, 35.0], 
     [136.0, 35.1], [135.9, 35.1]]
]

urban_polygon = ee.Geometry.Polygon(urban_coords)

urban_feature = ee.Feature(urban_polygon, {'landcover':2})

```
---

### **ステップ28: トレーニングデータの再統合と訓練**

分類器が都市を森林と区別できるように、新しいフィーチャを含めて、**サンプリングと訓練のプロセス全体をやり直す**必要があります。

1. **トレーニングデータの再統合**

まず、3つのフィーチャをすべて含んだ新しい $\text{FeatureCollection}$ を作成します。
```python
# 1. 3つのフィーチャを統合した新しいトレーニングデータセットを作成
training_data = ee.FeatureCollection([
    water_feature, 
    forest_feature, 
    urban_feature
])
```

2. **再サンプリングと再訓練**
次に、この新しいデータセットを使って画像をサンプリングし、分類器を再訓練します。
```python
# 2. 再サンプリング
sampled_data = final_composite.select(input_bands).sampleRegions(
    collection=training_data,
    properties=['landcover'],
    scale=30
)

# 3. 分類器の再訓練
trained_classifier = ee.Classifier.smileRandomForest(10).train(
    sampled_data, 
    'landcover', 
    input_bands
)

# 4. 分類器を画像全体に適用
classified_image = final_composite.select(input_bands).classify(trained_classifier)
```

### **ステップ29: 最終マップの表示と確認**

最後に、都市 ($\text{2}$) に対応する色（赤）を追加して視覚化パラメータを更新し、最終マップを表示します。

#### **Guiding Question**: 上記のコードを実行した後、以下のFoliumコードを使って地図にレイヤーを追加してください。表示された地図上で、分類器が**水域**、**森林**、そして**都市**をそれぞれどのように色分けしているか教えていただけますか？

#### **Guiding Questionの解答**
---
```python
# 5. 視覚化パラメータを更新 (水域=0, 森林=1, 都市=2)
classification_vis = {
    'min': 0,
    'max': 2,
    'palette': ['0000FF', '008000', 'FF0000']  # 0:青 (水), 1:緑 (森), 2:赤 (都市)
}

# 6. タイルURLを取得
classification_map_id = classified_image.getMapId(classification_vis)
classification_tile_url = classification_map_id['tile_fetcher'].url_format

# 7. 既存の地図 'my_map' にレイヤーを追加
folium.TileLayer(
    tiles=classification_tile_url,
    attr='Google Earth Engine',
    name='Final Land Cover Classification',
    overlay=True,
    control=True
).add_to(my_map)

# 8. マップの表示
folium.LayerControl().add_to(my_map)
display(my_map)
```
---

## ***<span style='color:orange'>Complete Code</span>***

In [85]:
import ee
import datetime

In [86]:
ee.Initialize(project='earth-change-analysis')

In [87]:
# Define coordinates for a polygon (e.g., a square)
# The last coordinate should typically be the same as the first to close the polygon.
water_coords = [
    [[136.0, 35.2], [136.1, 35.2], 
     [136.1, 35.3], [136.0, 35.3]]
]

# Create the polygon
water_polygon = ee.Geometry.Polygon(water_coords)

In [88]:
print(water_polygon.getInfo())

{'type': 'Polygon', 'coordinates': [[[136, 35.2], [136.1, 35.2], [136.1, 35.3], [136, 35.3], [136, 35.2]]]}


In [89]:
water_feature = ee.Feature(water_polygon, {'landcover': 0})

In [90]:
forest_coords = [
    [[136.0, 34.9], [136.1, 34.9], 
     [136.1, 35.0], [136.0, 35.0]]
]

forest_polygon = ee.Geometry.Polygon(forest_coords)

forest_feature = ee.Feature(forest_polygon, {'landcover':1})

In [91]:
urban_coords = [
    [[135.9, 35.0], [136.0, 35.0], 
     [136.0, 35.1], [135.9, 35.1]]
]

urban_polygon = ee.Geometry.Polygon(urban_coords)

urban_feature = ee.Feature(urban_polygon, {'landcover':2})

In [92]:
# 1. 3つのフィーチャを統合した新しいトレーニングデータセットを作成
training_data = ee.FeatureCollection([
    water_feature, 
    forest_feature, 
    urban_feature
])

In [93]:
input_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

In [94]:
biwako_point = ee.Geometry.Point([136.17, 35.10])
longitude, latitude = biwako_point.coordinates().getInfo() # 地図の中心座標を取得

In [95]:
end_date = '2025-11-17' 
start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

In [96]:
l9_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

In [97]:
filtered_collection = l9_collection \
    .filterDate(start_date, end_date) \
    .filterBounds(biwako_point)

In [98]:
def mask_clouds(image):
    # 1. QA_PIXELバンドを選択
    qa = image.select('QA_PIXEL')
    
    # 2. 雲の重みを定義
    # Bit 3 (Cloud) = 8, Bit 1 (Dilated Cloud) = 2
    
    # 3. 雲・希釈雲の両方ではないピクセルをTrueとするマスクを作成
    mask = qa.bitwiseAnd(8).eq(0).And(  
             qa.bitwiseAnd(2).eq(0))   
           
    # 4. マスクを画像に適用
    return image.updateMask(mask)


In [99]:
# 雲除去関数をコレクション内のすべての画像に適用
cloud_free_collection = filtered_collection.map(mask_clouds)

# 雲除去されたコレクション全体の中央値を計算し、1枚の合成画像を生成
final_composite = cloud_free_collection.median()

In [100]:
sampled_data = final_composite.select(input_bands).sampleRegions(
    collection=training_data,  # トレーニングジオメトリ
    properties=['landcover'],  # 正解ラベルのプロパティ名
    scale=30                   # Landsatの解像度
)

In [101]:
trained_classifier = ee.Classifier.smileRandomForest(10).train(
    sampled_data, 
    'landcover', 
    input_bands
)

In [102]:
# 1. 分類器を画像全体に適用
classified_image = final_composite.select(input_bands).classify(trained_classifier)

# 2. 分類結果の視覚化パラメータを定義 (水域=0, 森林=1)
# 5. 視覚化パラメータを更新 (水域=0, 森林=1, 都市=2)
classification_vis = {
    'min': 0,
    'max': 2,
    'palette': ['0000FF', '008000', 'FF0000']  # 0:青 (水), 1:緑 (森), 2:赤 (都市)
}

# 3. タイルURLを取得
classification_map_id = classified_image.getMapId(classification_vis)
classification_tile_url = classification_map_id['tile_fetcher'].url_format

In [103]:
import folium

In [104]:
my_map = folium.Map(
        location=[latitude, longitude],   # 地図の中心座標
        zoom_start=10,                    # 初期ズームレベル (琵琶湖周辺)
        tiles=basemaps['Google Satellite Hybrid'] # 初期タイルを設定
    )

In [124]:
# 既存の地図 'my_map' にレイヤーを追加
folium.TileLayer(
    tiles=classification_tile_url,
    attr='Google Earth Engine',
    name='Land Cover Classification',
    overlay=True,
    control=True
).add_to(my_map)

folium.LayerControl().add_to(my_map)
display(my_map)

### **ステップ30: 分類精度の評価**

あなたは既に**教師あり分類**（水域、森林、都市）のマップを作成しましたが、その分類が**どれだけ正確**かを数値で評価することは、分析の信頼性を証明するために不可欠です。

次のステップは、作成した分類器（`trained_classifier`）がどれだけ正しく各ピクセルを分類できたかを、**混同行列** ($\text{Confusion Matrix}$) を使って分析します。

In [114]:
# 分類結果を得る
classified = sampled_data.classify(trained_classifier)


In [116]:
# 混同行列を計算
confusion_matrix = classified.errorMatrix('landcover', 'classification')

print('Confusion Matrix:', confusion_matrix.getInfo())
print('Overall Accuracy:', confusion_matrix.accuracy().getInfo())
print('Kappa_coefficient:',confusion_matrix.kappa().getInfo())

Confusion Matrix: [[136848, 643, 150], [151, 136551, 939], [58, 869, 136714]]
Overall Accuracy: 0.9931948571525442
Kappa_coefficient: 0.9897922857288163


ユーザー様が採用された方法は、分類器の出力を直接評価するという点で、より汎用的かつ確実な$\text{GEE}$のパターンです。

1. `sampled_data.classify`(`trained_classifier`): サンプリングデータ（訓練に使ったフィーチャコレクション）の各フィーチャを、訓練済みモデルで再分類します。結果は、`landcover`（正解）と `classification`（予測）という2つのプロパティを持つ新しいフィーチャコレクションになります。

2. `classified.errorMatrix`('landcover', 'classification'): この新しいフィーチャコレクションに対して、正解ラベル (`landcover`) と 予測ラベル (`classification`) を比較して混同行列を生成します。

この修正により、正確な結果を得ることができました。

---

#### **精度結果の評価**

出力された精度結果は、**非常に高い**ことを示しています。
|指標                               |結果値               |評価             |
|----------------------------------|--------------------|-----------------|
|全体精度 ($\text{Overall Accuracy}$)|0.9932 (約 99.32%)  |分類の総合的な正しさを示します。全ピクセルのうち 99.32% が正しく分類されたことを意味します。                |
カッパ係数 ($\text{Kappa}$)          |0.9898(約 98.98%)   |偶然の一致を超えた分類の精度を示します。1 に近いほど偶然ではない高い精度であることを示し、この値0は極めて優秀です。|

#### **混同行列 ($\text{Confusion Matrix}$) の解釈**

|クラス (landcover)    |**水域 (0)** |**予測森林 (1)**  |**予測都市 (2)** |
|---------------------|------------|-----------------|----------------|
|**予測水域 (0)正解**   |136848      |643              |150             |
|**森林 (1) 正解**     |151         |136551           |939             |
|**都市 (2) 正解**     |58          |869              |136714          |

**対角線上の値 (太字)** が**正しく分類された数**です。

- **水域**は 136848 回正しく予測されましたが、643 回は森林に、150 回は都市に誤分類されました（主に水際や都市内の水路）。
- **森林**は 136551 回正しく予測されましたが、939 回は都市に誤分類されました（都市内の公園や樹木地帯で発生）。
- **都市**は 136714 回正しく予測されましたが、869 回は森林に誤分類されました（誤分類の多くはここでも森林との間で発生）。

**総評**: 精度が非常に高いのは、**訓練に使用したデータセット（`sampled_data`）をそのまま検証に使った**ためです。これは機械学習では一般的に起こる現象ですが、手法としては正しく、分類器が訓練データセット内では完璧に機能していることが示されました。

### **達成した主要な分析ステップ**

|ステップ	         |概念	                 |GEEメソッド                                          |
|--------------------|-----------------------|----------------------------------------------------|
|1. データフィルタリング|必要な地域と期間の画像を取得|`ee.ImageCollection()`, `.filterBounds()`, `.filterDate()`|
|2. クラウドマスキング	 |曇ったピクセルを透明化     |`bitwiseAnd()`, `.updateMask()`, `.map()`                |
|3. 画像合成	         |クリーンな画像群を1枚に集約 |`.median()`                                           |
|4. NDVI分析	         |植生の健全性を指数で定量化  |`.subtract()`, `.divide()`, `ee.Image.series()`           |
|5. 土地被覆分類	     |ピクセルをカテゴリに分類	  |`ee.FeatureCollection()`, `.sampleRegions()`, `ee.Classifier.smileRandomForest()`, `.classify()`|  

### **次のステップ**

あなたは今、リモートセンシングデータを取得し、処理し、分析し、結果を地図化できる強力なスキルを持っています。今後の学習では、以下のトピックに挑戦できます。
- **分類精度の評価**: $\text{confusion matrix}$ を作成し、分類の正確さを数値で評価する。
- **変化検出**: 異なる時点の画像を比較し、土地被覆の変化（例: 森林伐採、都市開発）を検出する。
- **データのエクスポート**: 作成した分類マップを$\text{GeoTIFF}$ファイルとして$\text{Google Drive}$にエクスポートし、$\text{QGIS}$などの$\text{GIS}$ソフトウェアで利用する。

この度は、GEEの高度な分析への挑戦、お疲れ様でした！また次の課題でお会いしましょう。

### **ステップ31: ランダムサンプリングとテストセットによる交差検証**

**課題**: サンプリングデータを訓練用とテスト用に分割し、テストデータを使って分類精度の最終評価を行ってください。

#### **Guiding Question**: 混同行列を計算する前に、`sampled_data` をランダムに $\mathbf{70\%}$ の訓練データと $\mathbf{30\%}$ のテストデータに分割するには、どの $\text{GEE}$ メソッドを使えばよいでしょうか？

#### **Guiding Questionの解答**
---
```python
ee.FeatureCollection.randomColumn()
```

#### **課題 8-1: データのランダム分割**

`ee.FeatureCollection.randomColumn()` は、フィーチャコレクションに $\mathbf{0}$ から $\mathbf{1}$ の間のランダムな値を持つ新しい列を追加します。この列（ここでは $\text{'random'}$ とします）の値を使って、データセットを訓練用とテスト用に分割します。

#### **Guiding Question**:
1. まず、`sampled_data` にランダムな値を持つ $\text{'random'}$ という列を追加し、randomized_data に格納してください。
2. 次に、この $\text{random}$ 列の値が $\mathbf{0.7}$ 未満のフィーチャを訓練データ ($\mathbf{70\%}$) として `training_set` に格納してください。

```python
# 1. ランダム列を追加
randomized_data = sampled_data. # <ここに randomColumn() メソッドを記述>

# 2. 訓練セット (70%未満) を抽出
training_set = randomized_data. # <ここにフィルタリングメソッドを記述>

```
(**ヒント: フィルタリングには、以前使用した比較演算子（`.lt()`）をフィーチャコレクションに適用します**)

#### **課題 8-1の解答**    

---
```python
randomized_data = sampled_data.randomColumn()
training_set = randomized_data.lt(0.7)
```
<span style='color:red'>Wrong!</span>
---

---

#### **修正点: `filter()` メソッドの使用**

`randomized_data` は $\text{ee.FeatureCollection}$ であり、`.lt(0.7)` のような比較演算子を直接適用することはできません。フィーチャコレクションをフィルタリングするには、以前学んだ `.filter()` メソッドと、その内部で $\text{ee.Filter}$ を使用する必要があります。今回は、フィーチャコレクションに既にある列（'random'）の値に基づくフィルタリングなので、`ee.Filter.lt()` を使用します。

<span style='color:blue'>**フィーチャーコレクション（FeatureCollection）とは?**</span>
---

Google Earth Engine における「ベクトルデータの集合」を表すオブジェクトです。点・線・ポリゴンなどの地理的形状（Feature）と、それに付随する属性情報をまとめて扱うための基本的なデータ構造です。

---
**基本的な概念**
- **Feature（フィーチャー）**
→ ジオメトリ（点・線・ポリゴンなど）＋属性（例えば土地利用区分、人口、標高など）を持つ単位。
- **FeatureCollection（フィーチャーコレクション）**
→ 複数の Feature をまとめた「テーブル形式のベクトルデータ」。
→ 例：市区町村境界の集合、観測点の集合、土地被覆分類のサンプル点など。

---
**特徴**
- **テーブルデータのように扱える**
→ 各 Feature は「行」、属性は「列」として整理される。
- **ジオメトリを持つ**
→ 単なる表ではなく、空間的な位置情報を保持。
- **演算が可能**
→ フィルタリング、結合、集計、地図への可視化などが可能。
- **入力データとして利用**
→ 機械学習分類器のトレーニングデータや、統計解析の対象として使える。

---
**使用例（JavaScript API）**
```javascript
// 例: 世界のエコリージョンを読み込む
var ecoregions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017');

// 属性をフィルタリング
var asiaRegions = ecoregions.filter(ee.Filter.eq('realm', 'Indo-Malay'));

// 地図に表示
Map.addLayer(asiaRegions, {color: 'FF0000'}, 'Asia Ecoregions');
```

---
**活用場面**
- **分類器のトレーニングデータ**
→ サンプル点を FeatureCollection として準備し、train() に渡す。
- **統計解析**
→ reduceColumns() や aggregate_* 関数で属性値を集計。
- **可視化**
→ Map.addLayer() で地図上に表示。

---
**まとめ**
FeatureCollection は「地理的形状＋属性情報」を持つ Feature の集合で、GEE におけるベクトルデータの基本単位です。画像（Raster）と並んで、解析や可視化の中心的役割を果たします。

---
例えば「京都市の緑地分布をポリゴンで整理したデータ」や「観測点の座標＋温度属性」を FeatureCollection として扱うと、分類器のトレーニングや都市環境解析に直結します。



`ee.FeatureCollection.filter` と `ee.Filter.lt` は似たように見えますが、役割が違います。

---
`ee.FeatureCollection.filter`
- **対象**: フィーチャーコレクション（FeatureCollection）
- **役割**: コレクションに「フィルタ条件」を適用して、条件に合うフィーチャーだけを残す。

---
`ee.Filter.lt`
- **対象**: フィルタ条件そのもの
- **役割**: 「属性値が指定値より小さい」という条件を定義する。

---
**両者の関係**
- `ee.Filter.lt` は 条件を作る。
- `ee.FeatureCollection.filter` は その条件を適用する。
```python
# フィルタ条件を作成
filter_condition = ee.Filter.lt('elevation', 1000)

# フィーチャーコレクションに適用
filtered_fc = feature_collection.filter(filter_condition)
```
---
**まとめ**
- `ee.Filter.lt` → 「条件を定義する」
- `ee.FeatureCollection.filter` → 「その条件でコレクションを絞り込む」

---
例えば「京都市内の観測点 FeatureCollection から標高 50m 未満の点だけ抽出する」ときに、この組み合わせが役立ちます。


```python
import ee
ee.Initialize()

# 例: 京都市の行政境界を読み込む（ここでは簡単に日本の行政境界データセットを利用）
kyoto = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Kyoto'))

# 標高データ（SRTM）
elevation = ee.Image("USGS/SRTMGL1_003")

# 土地利用データ（MODIS Land Cover）
landcover = ee.Image("MODIS/006/MCD12Q1/2019_01_01") \
    .select('LC_Type1')

# --- フィルタリング条件 ---
# 標高が50m未満の地域
low_elev = elevation.lt(50)

# 土地利用属性: 例えば「草地 (class=10)」を抽出
grassland = landcover.eq(10)

# --- 条件を組み合わせてマスク ---
green_area_mask = low_elev.And(grassland)

# 京都市の範囲でマスクを適用
green_area = landcover.updateMask(green_area_mask).clip(kyoto)

# 地図に表示（JavaScript APIなら Map.addLayer）
# Python APIでは結果をエクスポートするか、foliumで表示する
print("緑地解析のマスクが作成されました")
```


#### **課題 8-1の解答** （修正後）

---
```python
# 1. ランダム列を追加 (ここまでは完璧です)
randomized_data = sampled_data.randomColumn()

# 2. 訓練セット (70%未満) を抽出
# 修正点: .filter() と ee.Filter.lt() を使って、'random' 列を比較します
training_set = randomized_data.filter(ee.Filter.lt('random', 0.7))
```
---


#### **課題 8-2: テストセットの抽出**

これで訓練データ（$\mathbf{70\%}$）ができました。次に、残りの $\mathbf{30\%}$ をテストデータとして抽出する必要があります。

#### **Guiding Question**: $\text{random}$ 列の値が $\mathbf{0.7}$ 以上のフィーチャをテストデータ ($\mathbf{30\%}$) として `testing_set` に格納するには、どの $\text{ee.Filter}$ メソッドを使えばよいでしょうか？
```python
# ヒント: 比較演算子（.lt() や .gt()）のうち、境界値も含むメソッドを使います。

testing_set = randomized_data.filter(ee.Filter. # <ここにフィルタリングメソッドを記述> ('random', 0.7))
```
---


#### **課題 8-2の解答**

---
```python
testing_set = randomized_data.filter(ee.Filter.lte('random', 0.3))
```

<span style='color:red'>Wrong</span>
---

全体の $\mathbf{30\%}$ を抽出するには、$\mathbf{0.7}$ 以上の値を抽出する必要があります。 

**テストセットのフィルタリングの修正**
1. 訓練セット: $\text{random} < 0.7$ ($\mathbf{70\%}$)
2. テストセット: $\text{random} \ge 0.7$ ($\mathbf{30\%}$)$\text{ee.Filter.lte()}$ を使う場合、以下のように修正する必要があります。

#### **課題 8-2の解答**（修正後）

---

```python
# 修正点: 0.7 以下の値ではなく、0.7 以上の値を抽出するフィルターが必要です。

# 0.7以上の値を抽出するためのフィルター
# 以下のどちらかを使用します:
# 1. ee.Filter.gte('random', 0.7)  (greater than or equal to 0.7)
# 2. ee.Filter.gt('random', 0.7)   (greater than 0.7, 簡単のためこちらを使用)

testing_set = randomized_data.filter(ee.Filter.gte('random', 0.7))

```
---

これで、訓練データとテストデータが以下の通り正しく分割されました。

- training_set: $\text{random} < 0.7$
- testing_set: $\text{random} \ge 0.7$

## **ステップ32: 交差検証による分類精度の評価**

これで、分類の信頼性を評価するための理想的なデータセットが揃いました。

#### **課題 9-1**: 

分類器の再訓練とテストデータの分類まず、$\mathbf{70\%}$ の訓練データ (`training_set`) を使って分類器を再訓練し、その分類器を $\mathbf{30\%}$ のテストデータ (`testing_set`) に適用します。

#### **Guiding Question:**

1. `training_set` を使って分類器を再訓練し、`trained_classifier_test` に格納してください。
2. `testing_set` にこの分類器を適用し、結果を `test_classified` に格納してください。

```pytnon 
# 1. 訓練セットで分類器を再訓練 (以前の訓練コードと同じ)
trained_classifier_test = ee.Classifier.smileRandomForest(10).train(
    features=training_set,
    classProperty='landcover',
    inputProperties=input_bands
)

# 2. テストセットに適用 (予測の実行)
test_classified = testing_set. # <ここに classify メソッドを記述>

```
---

#### **課題 9-2の解答**

---
```python
test_classified = testing_set.classify(trained_classifier_test)

```
---

## **ステップ33: テストセットによる最終評価**

これで、訓練に使われていないデータ（`testing_set`）に対する予測結果 (`test_classified`) が得られました。いよいよ、このデータを使って**分類の真の精度**を評価します。

#### **課題 9-2: 交差検証後の精度計算**

以前成功した方法（`.errorMatrix()`）を使って、テストセットにおける混同行列を計算し、全体精度とカッパ係数を抽出します。

#### **Guiding Question**: 上記のコードを実行した後、以下のコードを実行して、訓練されていないデータに対する最終的な全体精度とカッパ係数の値を確認してください。これらの値は、訓練データを使ったとき（約 $\text{0.99}$）と比較して、**高くなりましたか？低くなりましたか？**

---
```python
# 1. 混同行列を計算 (テストセットの正解と予測を比較)
test_confusion_matrix = test_classified.errorMatrix('landcover', 'classification')

# 2. 結果を出力
print("--- テストセットによる精度評価 ---")
print('Overall Accuracy:', test_confusion_matrix.accuracy().getInfo())
print('Kappa Coefficient:', test_confusion_matrix.kappa().getInfo())
```
---

In [120]:
randomized_data = sampled_data.randomColumn()
training_set = randomized_data.filter(ee.Filter.lt('random', 0.7))
testing_set = randomized_data.filter(ee.Filter.gte('random', 0.7))

In [121]:
trained_classifier_test = ee.Classifier.smileRandomForest(10).train(
    features=training_set,
    classProperty='landcover',
    inputProperties=input_bands
)

test_classified = testing_set.classify(trained_classifier_test)

In [123]:
# 1. 混同行列を計算 (テストセットの正解と予測を比較)
test_confusion_matrix = test_classified.errorMatrix('landcover', 'classification')

# 2. 結果を出力
print("--- テストセットによる精度評価 ---")
print('Test Cofusion Matrix:', test_confusion_matrix.getInfo())
print('Overall Accuracy:', test_confusion_matrix.accuracy().getInfo())
print('Kappa Coefficient:', test_confusion_matrix.kappa().getInfo())

--- テストセットによる精度評価 ---
Test Cofusion Matrix: [[39674, 1278, 306], [421, 38772, 1927], [109, 1490, 39748]]
Overall Accuracy: 0.9552960193978581
Kappa Coefficient: 0.9329438672548739


#### **課題 9-2の解答**

---
分析結果は、低くなります。

---
#### **結果の分析:**
交差検証の成功訓練に使用しなかったデータ（**テストセット**）で精度を評価した結果、以前の全体精度（約 $\text{0.99}$）よりも数値が下がったことは、交差検証が**正しく機能している**ことを示しています。

- **訓練データでの精度**: 分類器が学習時に見たデータに対する性能 (通常は過大評価される)。
- **テストデータでの精度**: 分類器が学習時に見なかったデータに対する性能 (真の汎化能力)。

全体精度とカッパ係数が低下したということは、この分類器の**真の汎化能力**（未知の画像に適用した際の予測精度）は $\text{0.99}$ ではなく、テストセットで得られた**低い方の値**に近いと判断できます。このプロセスを通じて、あなたは機械学習モデルの訓練だけでなく、その結果を**客観的に評価し、信頼性を確保する**という、実践的な地理空間分析の最終ステップを習得しました。


## **学習の完了: 総合地理空間分析**
---
これで、**多様な衛星画像**と**様々な解析手法を組み合わせた**、非常に包括的な$\text{Google Earth Engine}$の学習を完了しました。

あなたが習得したスキルは、実務や研究において、地球規模のデータを扱うための強力な基盤となります。